<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%89_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
# -------------------------------------------------------------
# 區塊 A: 匯入、認證、API、Sheets 設定 (與您原來的程式碼相同)
# -------------------------------------------------------------
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
                gradio pandas beautifulsoup4 google-generativeai python-dateutil
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import userdata

In [73]:
# === Google Sheet 設定 ===
# 換成你自己的試算表網址與分頁名稱
SHEET_URL = "https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?usp=sharing"
try:
    gsheets = gc.open_by_url(SHEET_URL)
    print("✅ 成功開啟 Google 試算表！")
except Exception as e:
    print(f"❌ 開啟試算表失敗，請檢查 SHEET_URL 或認證步驟：{e}")

# 我們會自動確保三個分頁：tasks, pomodoro_logs, web_clips
TASKS_SHEET_NAME = "tasks"
LOGS_SHEET_NAME = "pomodoro_logs"
CLIPS_SHEET_NAME = "web_clips"
TIMEZONE = "Asia/Taipei"

def ensure_worksheet(sh, title, header):
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    # 若沒有表頭就補上
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = [
    "log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"
]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]

ws_tasks = ensure_worksheet(gsheets, TASKS_SHEET_NAME, TASKS_HEADER)
ws_logs  = ensure_worksheet(gsheets, LOGS_SHEET_NAME, LOGS_HEADER)
ws_clips = ensure_worksheet(gsheets, CLIPS_SHEET_NAME, CLIPS_HEADER)

✅ 成功開啟 Google 試算表！


In [74]:
# -------------------------------------------------------------
# 區塊 B: 核心函式定義 (已修正 tasks_data 邏輯)
# -------------------------------------------------------------

def tznow():
    return dt.now(gettz(TIMEZONE))

def read_data(ws, header, is_tasks=False):
    # (此函式邏輯正確，保持不變)
    if not is_tasks:
        df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
        if df is None or df.empty:
            return pd.DataFrame(columns=header)
        df = df.fillna("")
        for c in header:
            if c not in df.columns:
                df[c] = ""
        if "minutes" in df.columns:
            df["minutes"] = pd.to_numeric(df["minutes"], errors="coerce").fillna(0)
        return df[header]
    else:
        data = ws.get_all_records()
        for row in data:
            row_id = row.get("id", "")
            if not row_id: continue
            for col in ["est_min", "actual_min", "pomodoros"]:
                try:
                    row[col] = int(row.get(col) or 0)
                except ValueError:
                    row[col] = 0
        return data

def write_data(ws, data, header):
    # (此函式邏輯正確，保持不變)
    ws.clear()
    if isinstance(data, pd.DataFrame):
        if data.empty:
            ws.update([header])
            return
        df_out = data.copy()
        for c in df_out.columns:
            df_out[c] = df_out[c].astype(str)
        ws.update([header] + df_out.values.tolist())
    else:
        if not data:
            ws.update([header])
            return
        output_list = [header]
        for row_dict in data:
            row_list = [str(row_dict.get(col, "")) for col in header]
            output_list.append(row_list)
        ws.update(output_list)

# 修正後的 refresh_all 函式 (使用全局變數)
def refresh_all():
    global tasks_data, logs_df, clips_df # 宣告全局變數

    # 讀取任務：使用新的 read_data(..., is_tasks=True)
    tasks_data = read_data(ws_tasks, TASKS_HEADER, is_tasks=True)

    # 讀取 logs, clips：使用 read_data(..., is_tasks=False)
    logs_df = read_data(ws_logs, LOGS_HEADER, is_tasks=False)
    clips_df = read_data(ws_clips, CLIPS_HEADER, is_tasks=False)

    return (tasks_data.copy(), logs_df.copy(), clips_df.copy())

# 任務操作函式 (已修正為 tasks_data)

def add_task(task, priority, est_min, due_date, labels, notes, planned_for):
    global tasks_data
    _now = tznow().isoformat()
    new_task = { # 直接建立一個字典
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date or "",
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""
    }
    tasks_data.append(new_task)
    write_data(ws_tasks, tasks_data, TASKS_HEADER)
    return "✅ 已新增任務", tasks_data

def update_task_status(task_id_input, new_status):
    global tasks_data

    # 🚨 修正：直接使用傳入的值作為 task_id
    task_id = task_id_input

    if not task_id:
        return "⚠️ 請先選擇任務", tasks_data

    target_task = next((t for t in tasks_data if t.get("id") == task_id), None)

    if target_task is None:
        return "⚠️ 找不到任務", tasks_data

    target_task["status"] = new_status
    target_task["updated_at"] = tznow().isoformat()

    if new_status == "done" and not target_task.get("completed_at"):
        target_task["completed_at"] = tznow().isoformat()

    write_data(ws_tasks, tasks_data, TASKS_HEADER)
    return "✅ 狀態已更新", tasks_data

# 由於您之前有將 mark_done 包裝在 handle_update_task 內，現在可以直接確保邏輯順暢。
# 您的 Gradio 綁定應該是正確的：
# btn_done.click(
#     lambda task_c: handle_update_task(task_c, "done"),
#     inputs=[task_choice],
#     outputs=[msg_update, grid_tasks, task_choice_delete, task_choice, sel_task]
# )

def delete_task(task_id_input):
    global tasks_data

    # 🚨 修正：直接使用傳入的值作為 task_id
    task_id = task_id_input

    if not task_id:
        return "⚠️ 請先選擇要刪除的任務", tasks_data

    # 刪除任務：使用 List comprehension 重新建立列表
    new_tasks_data = [t for t in tasks_data if t.get("id") != task_id]

    if len(new_tasks_data) == len(tasks_data):
         return f"⚠️ 找不到 ID 為 {task_id} 的任務", tasks_data

    tasks_data = new_tasks_data
    write_data(ws_tasks, tasks_data, TASKS_HEADER)

    return f"✅ 任務 {task_id} 已刪除", tasks_data

def mark_done(task_choice):
    # 這裡 update_task_status 已經被修正，所以此 wrapper 函式正常
    return update_task_status(task_choice, "done")

# 修正：更新任務實際耗時 (必須使用 list/dict 邏輯)
def recalc_task_actuals(task_id):
    """根據 logs_df 回寫 actual_min 與 pomodoros"""
    global tasks_data, logs_df

    # 1. 篩選 Log (logs_df 仍為 DataFrame，此處不變)
    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    pomos = int(round(total_min / 25.0))

    # 2. 查找並更新任務 (List of Dicts 邏輯)
    target_task = next((t for t in tasks_data if t.get("id") == task_id), None)

    if target_task:
        target_task["actual_min"] = int(total_min)
        target_task["pomodoros"] = pomos
        target_task["updated_at"] = tznow().isoformat()

# 修正：列出任務選項 (必須使用 list/dict 邏輯)
def list_task_choices():
    global tasks_data
    if not tasks_data:
        return []

    # 顯示： [status] (P:priority) task — id
    def row_label(r):
        # 使用 .get() 確保欄位存在
        return f"[{r.get('status','N/A')}] (P:{r.get('priority','N/A')}) {r.get('task','Unknown')} — {r.get('id','N/A')}"

    # 回傳格式：[(顯示文字, 隱藏的 ID), ...]
    return [(row_label(t), t.get("id")) for t in tasks_data if t.get("id")]

In [75]:
# -------------------------------------------------------------
# 核心匯入/匯出功能函式
# -------------------------------------------------------------

# 助手函式：根據類型獲取對應的資料、表頭和 Sheet
def _get_data_context(data_type):
    # 🚨 確保所有全域變數都用 global 宣告
    global tasks_data, logs_data, TASKS_HEADER, LOGS_HEADER, ws_tasks, ws_logs

    # 確保資料結構是可用的 (假設已在程式開始時載入)
    if data_type == 'tasks':
        # 檢查 tasks_data 是否為 None 或空列表
        if not tasks_data:
            return None, TASKS_HEADER, ws_tasks, '任務'
        return tasks_data, TASKS_HEADER, ws_tasks, '任務'

    elif data_type == 'logs':
        # 檢查 logs_data 是否為 None 或空列表
        if not logs_data:
            return None, LOGS_HEADER, ws_logs, 'Log'
        return logs_data, LOGS_HEADER, ws_logs, 'Log'

    return None, None, None, None
# 1. 資料匯出函式
def export_data(data_list, data_header, data_name, file_format="csv"):
    if not data_list:
        return f"⚠️ {data_name} 資料是空的，無法匯出。", None

    df = pd.DataFrame(data_list, columns=data_header)

    # 使用 tempfile 創建臨時檔案，Gradio 讀取後會自動刪除
    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=f"_{data_name}.{file_format}") as tmp:
        file_path = tmp.name

        if file_format == "csv":
            # 使用 utf-8-sig 確保中文在 Excel 中不會亂碼
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
        elif file_format == "json":
            df.to_json(file_path, orient='records', indent=4, force_ascii=False)
        else:
            return f"⚠️ 不支援的檔案格式：{file_format}", None

    # 成功返回訊息和檔案路徑 (供 Gradio 下載)
    return f"✅ {data_name} 資料已匯出為 {file_format.upper()} 檔案。", file_path


# 2. 資料匯入函式 (核心邏輯)
def import_data(file_obj, data_header, ws_sheet):
    # 此函式只執行匯入的核心邏輯，回傳 (message, new_data_list)
    if file_obj is None:
        return "⚠️ 請先上傳檔案", None

    file_path = file_obj.name
    file_ext = os.path.splitext(file_path)[1].lower()

    try:
        if file_ext == '.csv':
            imported_df = pd.read_csv(file_path)
        elif file_ext == '.json':
            imported_df = pd.read_json(file_path)
        else:
            return "⚠️ 不支援的檔案格式，請上傳 .csv 或 .json 文件。", None

        # 欄位驗證
        missing_cols = [col for col in data_header if col not in imported_df.columns]
        if missing_cols:
            return f"⚠️ 匯入失敗：缺少必要欄位 {', '.join(missing_cols)}", None

        # 確保資料框架只包含必要的欄位，並按正確的順序排列
        imported_df = imported_df[data_header]

        # 轉換回列表字典格式
        new_data_list = imported_df.to_dict('records')

        # 寫回 Google Sheet
        write_data(ws_sheet, new_data_list, data_header)

        return f"✅ 資料已成功匯入 {len(new_data_list)} 筆，並同步至 Google Sheet。", new_data_list

    except Exception as e:
        return f"❌ 匯入發生錯誤：{e}", None


# 3. Gradio 匯入 Wrapper (任務專用)
def handle_import_tasks(file_obj):
    global tasks_data, tasks_data_bak # tasks_data_bak 假設用於任務備份/歷史紀錄

    # 執行核心匯入
    msg, new_data = import_data(file_obj, TASKS_HEADER, ws_tasks)

    # 如果匯入成功
    if new_data is not None:
        tasks_data = new_data # 更新全域變數
        # 更新任務 Grid 和所有任務下拉選單
        new_choices = list_task_choices(tasks_data)

        # 任務匯入需要更新：訊息、任務表格、所有下拉選單（task_choice, sel_task, task_choice_delete）
        return msg, tasks_data, new_choices, new_choices, new_choices

    # 如果匯入失敗，回傳舊的狀態 (避免錯誤清除介面)
    current_choices = list_task_choices(tasks_data)
    return msg, tasks_data, current_choices, current_choices, current_choices


# 4. Gradio 匯入 Wrapper (Log 專用)
def handle_import_logs(file_obj):
    global logs_data

    # 執行核心匯入
    msg, new_data = import_data(file_obj, LOGS_HEADER, ws_logs)

    # 如果匯入成功
    if new_data is not None:
        logs_data = new_data # 更新全域變數
        # Log 匯入只需要更新：訊息、Log 表格
        return msg, logs_data

    # 如果匯入失敗，回傳舊的狀態
    return msg, logs_data
def export_wrapper(data_type, file_format):
    """提取正確的參數並呼叫 export_data"""
    # _get_data_context 返回: (data_list, data_header, ws_sheet, data_name)
    data_list, data_header, _, data_name = _get_data_context(data_type)

    # 🚨 檢查 data_list 是否為 None/空
    if data_list is None or not data_list:
        return f"⚠️ {data_name} 資料目前是空的，無法匯出。請先確認 Google Sheet 中是否有資料。", None

    # 呼叫 export_data(data_list, data_header, data_name, file_format)
    return export_data(data_list, data_header, data_name, file_format)

In [76]:
# Pomodoro 邏輯
_active_sessions = {}

def start_phase(task_id, phase, cycles):
    if not task_id: return "⚠️ 請先選擇任務"
    _active_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles) if cycles else 1
    }
    return f"▶️ 已開始：{phase}（task: {task_id}）"

def end_phase(task_id, note):
    global logs_df, tasks_data
    if task_id not in _active_sessions:
        return "⚠️ 尚未開始任何階段"
    sess = _active_sessions.pop(task_id)
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)
    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": int(sess["cycles"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_data(ws_logs, logs_df, LOGS_HEADER)

    # 回填任務
    if sess["phase"] == "work":
        recalc_task_actuals(task_id) # 呼叫修正後的 recalc_task_actuals
        write_data(ws_tasks, tasks_data, TASKS_HEADER) # 使用 tasks_data

    return f"⏹️ 已結束：{sess['phase']}，紀錄 {minutes} 分鐘"

In [77]:
# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [78]:
# 修正：AI 計畫 (tasks_df -> tasks_data 轉換)
def generate_today_plan():
    global tasks_data
    # 1. 轉換為 DataFrame 進行複雜篩選和排序
    tasks_df_temp = pd.DataFrame(tasks_data, columns=TASKS_HEADER)

    # 以下使用 DataFrame 邏輯
    today = tznow().date().isoformat()
    cand = tasks_df_temp[
        ((tasks_df_temp["due_date"]==today) | (tasks_df_temp["planned_for"].str.lower()=="today")) &
        (tasks_df_temp["status"]!="done")
    ].copy()

    if cand.empty:
        return "📭 今天沒有標記的任務。請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today。"

    # 先依 priority（H>M>L）+ est_min 排序
    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    # 嘗試 Gemini (邏輯不變)
    api_key = os.environ.get("GEMINI_API_KEY","").strip()
    plan_md = ""
    # ... (Gemini 邏輯略，保持原樣)
    if api_key:
         genai.configure(api_key=api_key)
         sys_prompt = (
             "你是一位任務規劃助理。請把輸入的任務（含估時與優先級）排成三段：morning、afternoon、evening，"
             "並給出每段的重點、順序、每項的時間預估與備註。總時數請大致符合任務估時總和。"
             "回傳以 Markdown 條列，格式：\n"
             "### Morning\n- [任務ID] 任務名稱（預估 xx 分）— 備註\n..."
             "### Afternoon\n...\n### Evening\n...\n"
         )
         items = []
         for _, r in cand.iterrows():
             items.append({
                 "id": r["id"], "task": r["task"], "est_min": int(r["est_min"]),
                 "priority": r["priority"]
             })
         user_content = json.dumps({"today": today, "tasks": items}, ensure_ascii=False)
         try:
             model = genai.GenerativeModel("gemini-1.5-flash")
             resp = model.generate_content(sys_prompt + "\n\n" + user_content)
             plan_md = resp.text
         except Exception as e:
             plan_md = f"⚠️ Gemini 失敗：{e}\n\n改用規則式規劃。"
    else:
        plan_md = "🔧 未設定 GEMINI_API_KEY，使用規則式規劃。\n\n"

    # 規則式：把高優先任務平均切到上午/下午/晚上
    buckets = {"morning": [], "afternoon": [], "evening": []}
    total = len(cand)
    for i, (_, r) in enumerate(cand.iterrows()):
        if i % 3 == 0:
            buckets["morning"].append(r)
        elif i % 3 == 1:
            buckets["afternoon"].append(r)
        else:
            buckets["evening"].append(r)

    def sec_md(name, rows):
        if not rows: return f"### {name.title()}\n（無）\n"
        lines = [f"### {name.title()}"]
        for r in rows:
            lines.append(f"- [{r['id']}] {r['task']}（預估 {int(r['est_min'])} 分，P:{r['priority']}）")
        return "\n".join(lines) + "\n"

    rule_md = sec_md("morning", buckets["morning"]) + "\n" + \
              sec_md("afternoon", buckets["afternoon"]) + "\n" + \
              sec_md("evening", buckets["evening"])

    return (plan_md + "\n---\n" + rule_md).strip()
    # 修正：今日完成率 (使用 List of Dicts 邏輯)
def today_summary():
    global tasks_data

    today = tznow().date().isoformat()

    # 篩選計畫任務 (List Comprehension)
    planned = [
        t for t in tasks_data
        if t.get("due_date") == today or t.get("planned_for", "").lower() == "today"
    ]
    done = [t for t in planned if t.get("status") == "done"]

    total = len(planned)
    done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    return f"📅 今日計畫任務：{total}；✅ 完成：{done_n}；📈 完成率：{rate:.1f}%"

In [79]:
# 爬蟲和加入任務 (已修正)
def crawl(url, selector, mode, limit):
    # (此函式邏輯不變，回傳 DataFrame)
    # ... (原 crawl 函式內容)
    try:
        resp = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0"})
        resp.raise_for_status()
    except Exception as e:
        return pd.DataFrame(columns=CLIPS_HEADER), f"⚠️ 請求失敗：{e}"

    soup = BeautifulSoup(resp.text, "html.parser")
    nodes = soup.select(selector)
    rows = []
    for i, n in enumerate(nodes[:int(limit) if limit else 20]):
        text = n.get_text(strip=True) if mode in ("text","both") else ""
        href = n.get("href") if mode in ("href","both") else ""
        # 相對連結處理
        if href and href.startswith("/"):
            from urllib.parse import urljoin
            href = urljoin(url, href)
        rows.append({
            "clip_id": str(uuid.uuid4())[:8],
            "url": url,
            "selector": selector,
            "text": text,
            "href": href,
            "created_at": tznow().isoformat(),
            "added_to_task": ""
        })
    df = pd.DataFrame(rows, columns=CLIPS_HEADER)
    return df, f"✅ 擷取 {len(df)} 筆"


def add_clips_as_tasks(clip_ids, default_priority, est_min):
    global clips_df, tasks_data # 注意：tasks_df 換成 tasks_data
    if not clip_ids:
        return "⚠️ 請先勾選要加入的爬蟲項目", clips_df, tasks_data

    # 1. 篩選 Clips (DataFrame 邏輯不變)
    sel = clips_df[clips_df["clip_id"].isin(clip_ids)]
    _now = tznow().isoformat()
    new_tasks = []

    # 2. 建立新任務 (List of Dicts)
    for _, r in sel.iterrows():
        title = r["text"] or r["href"] or "（未命名）"
        note = f"來源：{r['url']}\n選擇器：{r['selector']}\n連結：{r['href']}"
        new_tasks.append({
            "id": str(uuid.uuid4())[:8],
            "task": title[:120],
            "status": "todo",
            "priority": default_priority or "M",
            "est_min": int(est_min) if est_min else 25,
            "start_time": "",
            "end_time": "",
            "actual_min": 0,
            "pomodoros": 0,
            "due_date": "",
            "labels": "from:crawler",
            "notes": note,
            "created_at": _now,
            "updated_at": _now,
            "completed_at": "",
            "planned_for": ""
        })

    if new_tasks:
        # 3. 追加任務到 tasks_data (List of Dicts 邏輯)
        tasks_data.extend(new_tasks)

        # 4. 標記已加入 (DataFrame 邏輯不變)
        clips_df.loc[clips_df["clip_id"].isin(clip_ids), "added_to_task"] = "yes"

        # 5. 寫回 Sheets (使用新的 write_data)
        write_data(ws_tasks, tasks_data, TASKS_HEADER)
        write_data(ws_clips, clips_df, CLIPS_HEADER)

        return f"✅ 已加入 {len(new_tasks)} 項為任務", clips_df, tasks_data

    return "⚠️ 無可加入項目", clips_df, tasks_data

In [80]:
# -------------------------------------------------------------
# 區塊 C: 資料初始化 (必須在 Gradio 介面定義之前)
# -------------------------------------------------------------

# 執行初始化並載入資料到全域變數
tasks_data, logs_df, clips_df = refresh_all()

In [81]:
# -------------------------------------------------------------
# 區塊 D: Gradio 輔助/包裝函式
# -------------------------------------------------------------

# 輔助函式：將 List of Dicts 轉換成 Gradio Dataframe 喜歡的 DataFrame
def tasks_to_df(data):
    return pd.DataFrame(data, columns=TASKS_HEADER)

def handle_add_task(task, p, e, d, l, n, pf):
    """處理新增任務的包裝函式"""
    msg, tasks_data_list = add_task(task, p, e, d, l, n, pf)
    tasks_dataframe = tasks_to_df(tasks_data_list)
    # 確保任務選項下拉選單也更新
    new_choices = list_task_choices()
    return msg, tasks_dataframe, new_choices, new_choices, new_choices


def handle_update_task(task_choice, new_status):
    """處理更新狀態的包裝函式"""
    msg, tasks_data_list = update_task_status(task_choice, new_status)
    tasks_dataframe = tasks_to_df(tasks_data_list)
    new_choices = list_task_choices()
    return msg, tasks_dataframe, new_choices, new_choices, new_choices


def handle_delete_task(task_choice_delete):
    """處理刪除任務的包裝函式"""
    msg, tasks_data_list = delete_task(task_choice_delete)
    tasks_dataframe = tasks_to_df(tasks_data_list)
    new_choices = list_task_choices()
    return msg, tasks_dataframe, new_choices, new_choices, new_choices


def handle_refresh():
    """處理重新整理按鈕：從 Sheet 讀取最新資料並更新所有介面元件"""

    # 1. 執行同步操作 (從 Google Sheet 讀取最新資料)
    tasks_data_list, logs_df_data, clips_df_data = refresh_all()

    # 2. 轉換 tasks_data 的格式
    tasks_df_display = tasks_to_df(tasks_data_list)

    # 3. 重新計算任務選擇列表與摘要
    new_choices = list_task_choices()
    new_summary = today_summary()

    msg = f"🔄 成功從 Google Sheet 重新整理資料。最後更新時間：{tznow().strftime('%H:%M:%S')}"

    # 回傳順序必須與 outputs 一致：[msg, grid_tasks, grid_logs, grid_clips, task_choice, summary]
    return (
        msg,
        tasks_df_display,
        logs_df_data,
        clips_df_data,
        new_choices, # 任務選項列表 (用於三個 Dropdown)
        new_summary
    )

def handle_end_phase(task_id, note, phase):
    """處理結束工作/休息的包裝函式"""
    global tasks_data # 需要使用 global 才能更新

    msg = end_phase(task_id, note)

    # 刷新任務清單和選項
    # 由於 end_phase 內部已經呼叫了 write_data(ws_tasks, tasks_data, TASKS_HEADER)
    # 這裡直接回傳最新的 tasks_data 即可

    tasks_dataframe = tasks_to_df(tasks_data)
    new_choices = list_task_choices()

    # Gradio 回傳: [訊息, 任務清單, 日誌清單, 任務選項]
    return msg, tasks_dataframe, logs_df, new_choices

def handle_add_clips(clip_ids_str, pr, est):
    """處理加入爬蟲結果為任務的包裝函式"""
    ids = [c.strip() for c in (clip_ids_str or "").split(",") if c.strip()]
    msg, new_clips_df, new_tasks_data = add_clips_as_tasks(ids, pr, est)

    tasks_dataframe = tasks_to_df(new_tasks_data)
    new_choices = list_task_choices()

    return msg, new_clips_df, tasks_dataframe, new_choices, new_choices, new_choices

In [82]:
# -------------------------------------------------------------
# 區塊 E: Gradio 介面
# -------------------------------------------------------------
def handle_end_break_phase_wrapper(task_id, note):
    """
    處理結束休息的包裝函式。
    呼叫 handle_end_phase，並對 grid_tasks 和 sel_task 返回 gr.update()。
    """
    # handle_end_phase 返回: (msg, tasks_dataframe, logs_df, new_choices)
    msg, _, logs_df_ret, new_choices_ret = handle_end_phase(task_id, note, "break")

    # 回傳順序對應 outputs=[msg_pomo, grid_tasks, grid_logs, sel_task]
    return msg, gr.update(), logs_df_ret, gr.update() # 休息階段不更新任務清單和下拉選單
with gr.Blocks(title="待辦清單＋番茄鐘＋AI 計畫（Sheet/Gradio/爬蟲）") as demo:
    gr.Markdown("# ✅ 待辦清單與番茄鐘（Google Sheet＋Gradio＋Crawler＋AI 計畫）")

    # 放置一個用於顯示狀態訊息的元件
    msg_app = gr.Markdown(value="App 已啟動。請點擊 '重新整理' 獲取最新資料。", visible=True)

    with gr.Row():
        btn_refresh = gr.Button("🔄 重新整理（Sheet → App）")
        out_summary = gr.Markdown(today_summary()) # 初始值使用修正後的 today_summary()

    with gr.Tab("Tasks"):
        with gr.Row():
            with gr.Column(scale=2):
                task = gr.Textbox(label="任務名稱", placeholder="寫 HW3 報告 / 修正 SQL / …")
                priority = gr.Dropdown(["H","M","L"], value="M", label="優先級")
                est_min = gr.Number(value=25, label="預估時間（分鐘）", precision=0)
                due_date = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
                labels = gr.Textbox(label="標籤（逗號分隔，可空白）")
                notes = gr.Textbox(label="備註（可空白）")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃歸屬")
                btn_add = gr.Button("➕ 新增任務")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                # 修正：初始化值使用轉換後的 tasks_data
                grid_tasks = gr.Dataframe(value=tasks_to_df(tasks_data), label="任務清單（直接從 Sheet 來）", interactive=False)

        # === 刪除任務區 ===
        with gr.Row():
            with gr.Column(scale=2):
                # 修正：初始化值使用修正後的 list_task_choices
                task_choice_delete = gr.Dropdown(
                    choices=list_task_choices(),
                    label="選取要刪除的任務",
                    interactive=True
                )
            with gr.Column(scale=1):
                delete_btn = gr.Button("🗑️ 刪除任務", variant="stop", scale=1)
            msg_delete = gr.Markdown()

        with gr.Row():
            # 修正：初始化值使用修正後的 list_task_choices
            task_choice = gr.Dropdown(choices=list_task_choices(), label="選取任務（用於更新）")
            new_status = gr.Dropdown(["todo","in-progress","done"], value="in-progress", label="更新狀態")
            btn_update = gr.Button("✏️ 更新狀態")
            btn_done = gr.Button("✅ 直接標記完成")
            msg_update = gr.Markdown()

    with gr.Tab("Pomodoro"):
        with gr.Row():
            # 修正：初始化值使用修正後的 list_task_choices
            sel_task = gr.Dropdown(choices=list_task_choices(), label="選擇任務")
            cycles = gr.Number(value=1, precision=0, label="番茄數（僅作紀錄）")
        with gr.Row():
            btn_start_work = gr.Button("▶️ 開始工作")
            note_work = gr.Textbox(label="工作備註（可空白）")
            btn_end_work = gr.Button("⏹️ 結束工作並記錄")
        with gr.Row():
            btn_start_break = gr.Button("🍵 開始休息")
            note_break = gr.Textbox(label="休息備註（可空白）")
            btn_end_break = gr.Button("⏹️ 結束休息並記錄")
        msg_pomo = gr.Markdown()
        grid_logs = gr.Dataframe(value=logs_df, label="番茄鐘紀錄", interactive=False)

    with gr.Tab("AI Plan"):
        gr.Markdown("把**今天的任務**排成 **morning / afternoon / evening** 三段行動計畫。若未設 GEMINI_API_KEY，會用規則式。")
        btn_plan = gr.Button("🧠 產生今日計畫")
        out_plan = gr.Markdown()

    with gr.Tab("Crawler"):
        url = gr.Textbox(label="目標 URL", placeholder="https://example.com")
        selector = gr.Textbox(label="CSS Selector", placeholder="a.news-item / h2.title / div.card a")
        mode = gr.Radio(["text","href","both"], value="text", label="擷取內容")
        limit = gr.Number(value=20, precision=0, label="最多擷取幾筆")
        btn_crawl = gr.Button("🕷️ 開始擷取")
        msg_crawl = gr.Markdown()
        grid_clips = gr.Dataframe(value=clips_df, label="擷取結果（會同步寫入 Sheet）", interactive=True)
        clip_ids = gr.Textbox(label="要加入任務的 clip_id（多個以逗號分隔）")
        default_priority = gr.Dropdown(["H","M","L"], value="L", label="新增任務優先級")
        clip_est = gr.Number(value=25, precision=0, label="新增任務預估分鐘")
        btn_add_clips = gr.Button("➕ 將勾選的擷取項目加入為任務")
        msg_add_clips = gr.Markdown()

    with gr.Tab("Summary"):
        btn_summary = gr.Button("📊 重新計算今日完成率")
        out_summary2 = gr.Markdown(value=today_summary())
    with gr.Tab("Data IO"):
        gr.Markdown("### 🗃️ 資料匯入/匯出管理")

        msg_io = gr.Markdown("選擇操作類型與檔案格式。")

        # 匯出區塊
        gr.Markdown("#### 📤 匯出資料 (Download)")
        with gr.Row():
            data_type_export = gr.Radio(
                ["tasks", "logs"],
                label="選擇匯出資料類型",
                value="tasks"
            )
            file_format_export = gr.Radio(
                ["csv", "json"],
                label="選擇檔案格式",
                value="csv"
            )
        with gr.Row():
            btn_export = gr.Button("點擊匯出")
            # 使用 gr.File 作為輸出元件，Gradio 會自動生成下載連結
            file_export = gr.File(label="下載檔案", file_count="single", interactive=False)

        gr.Markdown("---")

        # 匯入區塊
        gr.Markdown("#### 📥 匯入資料 (Upload / **注意：將覆蓋 Google Sheet 紀錄！**)")

        # 任務匯入
        gr.Markdown("##### 匯入任務紀錄")
        with gr.Row():
            file_import_tasks = gr.File(label="上傳任務 CSV/JSON 檔案", file_count="single", file_types=['.csv', '.json'])
            btn_import_tasks = gr.Button("匯入任務 (覆蓋)")

        # Log 匯入
        gr.Markdown("##### 匯入 Log 紀錄")
        with gr.Row():
            file_import_logs = gr.File(label="上傳 Log CSV/JSON 檔案", file_count="single", file_types=['.csv', '.json'])
            btn_import_logs = gr.Button("匯入 Log (覆蓋)")
    # === 綁定動作 ===

    # 修正：綁定重新整理按鈕，回傳所有更新的元件
    btn_refresh.click(
        handle_refresh,
        inputs=None,
        outputs=[msg_app, grid_tasks, grid_logs, grid_clips, task_choice, out_summary] # 這裡只放 out_summary
        # outputs=[msg_app, grid_tasks, grid_logs, grid_clips, task_choice_delete, task_choice, sel_task, out_summary] # (完整輸出)
    )

    # 調整後的 Gradio 綁定範例
    btn_add.click(
        handle_add_task,
        inputs=[task, priority, est_min, due_date, labels, notes, planned_for],
        outputs=[msg_add, grid_tasks, task_choice_delete, task_choice, sel_task] # 輸出更新 Dropdown
    )

    btn_update.click(
        lambda task_c, new_s: handle_update_task(task_c, new_s),
        inputs=[task_choice, new_status],
        outputs=[msg_update, grid_tasks, task_choice_delete, task_choice, sel_task]
    )

    # === 綁定刪除任務功能 ===
    delete_btn.click(
        fn=handle_delete_task,
        inputs=[task_choice_delete],
        outputs=[msg_delete, grid_tasks, task_choice_delete, task_choice, sel_task] # 輸出更新 Dropdown
    )

    btn_done.click(
        lambda task_c: handle_update_task(task_c, "done"),
        inputs=[task_choice],
        outputs=[msg_update, grid_tasks, task_choice_delete, task_choice, sel_task]
    )

    # 修正：End Phase 時要更新任務清單和 Log 清單
    btn_end_work.click(
        lambda task_id, note: handle_end_phase(task_id, note, "work"),
        inputs=[sel_task, note_work],
        outputs=[msg_pomo, grid_tasks, grid_logs, sel_task]
    )

    # 修正：End Break 綁定 (使用新的 Wrapper，並修正 outputs 列表)
    btn_end_break.click(
        handle_end_break_phase_wrapper,
        inputs=[sel_task, note_break],
        # outputs 必須是元件物件
        outputs=[msg_pomo, grid_tasks, grid_logs, sel_task]
    )
    # Start Phase 不變，因為只更新狀態訊息
    btn_start_work.click(
        start_phase, inputs=[sel_task, gr.State("work"), cycles], outputs=[msg_pomo]
    )
    btn_start_break.click(
        start_phase, inputs=[sel_task, gr.State("break"), cycles], outputs=[msg_pomo]
    )

    btn_plan.click(generate_today_plan, outputs=[out_plan])
# 匯出綁定 (使用 export_data 的 Wrapper)
    btn_export.click(
    export_wrapper, # 🚨 使用修正後的包裝函式
    inputs=[data_type_export, file_format_export],
    outputs=[msg_io, file_export]
)

    # 任務匯入綁定
    # 輸出元件必須包含所有需要更新的任務相關元件
    btn_import_tasks.click(
        handle_import_tasks,
        inputs=[file_import_tasks],
        # outputs: 訊息, 任務表格, 刪除下拉選單, 新增下拉選單, Pomodoro下拉選單
        outputs=[msg_io, grid_tasks, task_choice_delete, task_choice, sel_task]
    )

    # Log 匯入綁定
    btn_import_logs.click(
        handle_import_logs,
        inputs=[file_import_logs],
        # outputs: 訊息, Log 表格
        outputs=[msg_io, grid_logs]
    )
    def _crawl_and_save(u, s, m, l):
        df, msg = crawl(u, s, m, l)
        global clips_df
        if not df.empty:
            clips_df = pd.concat([clips_df, df], ignore_index=True)
            write_data(ws_clips, clips_df, CLIPS_HEADER) # 使用修正後的 write_data
        return msg, clips_df

    btn_crawl.click(_crawl_and_save, inputs=[url, selector, mode, limit], outputs=[msg_crawl, grid_clips])

    # 修正：爬蟲新增任務後，也要更新所有任務相關的元件
    btn_add_clips.click(
        handle_add_clips,
        inputs=[clip_ids, default_priority, clip_est],
        outputs=[msg_add_clips, grid_clips, grid_tasks, task_choice_delete, task_choice, sel_task]
    )

    btn_summary.click(today_summary, outputs=[out_summary2])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a2c02f428b6797a52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
